In [1]:
import torch.nn as nn
from torchvision.models import resnet18, resnet34, inception_v3, densenet121
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.cuda import is_available
from torch.autograd import Variable
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch
from torch import optim
import numpy as np
import glob
import os

# **Preparing Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!7z x /content/drive/MyDrive/Colab\ Notebooks/Data\ Science/Learning/PyTorch/dogs-vs-cats.zip
!7z x /content/train.zip
!7z x /content/test1.zip 

In [3]:
path = '/content/dogs-vs-cats'
os.mkdir(path)
os.rename('/content/train', os.path.join(path, 'train'))
os.rename('/content/test1', os.path.join(path, 'test1'))
os.mkdir(os.path.join(path, 'valid'))
files = glob.glob(os.path.join(path, 'train/*.jpg'))
no_of_images = len(files)
shuffle = np.random.permutation(no_of_images)
valid_size = int(3*int(no_of_images / 10))
for i in ['train', 'valid']:
  for folder in ['dog', 'cat']:
    os.mkdir(os.path.join(path, i, folder))
for i in shuffle[:valid_size]:
  image = files[i].split('/')[-1]
  folder = image.split('.')[0]
  os.rename(files[i], os.path.join(path, 'valid', folder, image))
for i in shuffle[valid_size:]:
  image = files[i].split('/')[-1]
  folder = image.split('.')[0]
  os.rename(files[i], os.path.join(path, 'train', folder, image))

In [4]:
class ResNetBasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.stride = stride
  def forward(self, x):
    residual = x
    out = self.conv1(x)
    out = F.relu(self.bn1(out), inplace = True)
    out = self.conv2(out)
    out += residual
    return F.relu(out)

# **Creating and Training: Resnet Model**

In [ ]:
resnet = resnet18(pretrained = False)
data_transform = transforms.Compose([transforms.Resize((299, 299)), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train = ImageFolder(os.path.join(path, 'train'), transform = data_transform)
val = ImageFolder(os.path.join(path, 'valid'), transform = data_transform)
classes = 2
train_loader = DataLoader(train, batch_size = 32, num_workers = 3, shuffle = False)
val_loader = DataLoader(val, batch_size = 32, num_workers = 3, shuffle = False)


In [ ]:
#pretrained resnet
resnet_34 = resnet34(pretrained = True)
if is_available():
  resnet_34 = resnet_34.to('cuda')
  resnet_34 = nn.Sequential(*list(resnet_34.children())[:-1])
for p in resnet_34.parameters():
  p.requires_grad = False
# extracting convolutional features
trn_labels = []
trn_features = []
for d,la in train_loader:
  o = resnet_34(Variable(d.to("cuda")))
  o = o.view(o.size(0), -1)
  trn_labels.extend(la)
  trn_features.extend(o.cpu().data)
val_labels = []
val_features = []
for d,la in val_loader:
  o = resnet_34(Variable(d.to("cuda")))
  o = o.view(o.size(0), -1)
  val_labels.extend(la)
  val_features.extend(o.cpu().data)

class FeaturesDataset(Dataset):
  def __init__(self, featlst, labellst):
    self.featlst = featlst
    self.labellst = labellst
  def __getitem__(self, index):
    return (self.featlst[index], self.labellst[index])
  def __len__(self):
    return len(self.labellst)

trn_feat_dset = FeaturesDataset(trn_features, trn_labels)
val_feat_dset = FeaturesDataset(val_features, val_labels)
trn_feat_loader = DataLoader(trn_feat_dset, batch_size = 64, shuffle = True)
val_feat_loader = DataLoader(val_feat_dset, batch_size = 64)

class FullyConnectedModel(nn.Module):
  def __init__(self, in_size, out_size):
    super().__init__()
    self.fc = nn.Linear(in_size, out_size)
  def forward(self, inp):
    out = self.fc(inp)
    return out

fc_in_size = 8192
fc = FullyConnectedModel(fc_in_size, classes)
if is_available:
  fc = fc.cuda()
optimizer = optim.SGD(fc.parameters(), lr = 0.01, momentum = 0.5)

def fit(epoch, model, data_loader, phase = 'training', volatile = False):
    if phase == 'training':
      model.train()
    if phase == 'validation':
      model.eval()
      volatile = True
    running_loss = 0.0
    running_correct = 0
    for batch_idx, (data, target) in enumerate(data_loader):
      if is_available():
        data, target = data.to("cuda"), target.to("cuda")
      data, target = Variable(data, volatile), Variable(target)
      if phase == 'training':
        optimizer.zero_grad()
      output = model(data)
      loss = F.nll_loss(output, target)
      running_loss += F.nll_loss(output, target, size_average = False).item()      
      preds = output.data.max(dim = 1, keepdim = True)[1]
      running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
      if phase == 'traning':
        loss.backward()
        optimizer.step()
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. *running_correct/len(data_loader.dataset)
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss, accuracy

train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []
for epoch in range(1, 10):
  epoch_loss, epoch_accuracy = fit(epoch, fc, trn_feat_loader, phase = 'training')
  val_epoch_loss, val_epoch_accuracy = fit(epoch, fc, val_feat_loader, phase = 'validation')
  train_losses.append(epoch_loss)
  train_accuracy.append(epoch_accuracy)
  val_losses.append(val_epoch_loss)
  val_accuracy.append(val_epoch_accuracy)

KeyboardInterrupt: ignored

# **Building and Training: Inception Model**

In [ ]:
class BasicConv2d(nn.Module):
  def __init__(self, in_channels, out_channels, **kwargs):
    super(BasicConv2d, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, bias = False, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels)
  def forward(self, x):
    x = self.conv(x)
    x = self.bn(x)
    return F.relu(x, inplace = True)

class InceptionBasicBlock(nn.Module):
  def __init__(self, in_channels, pool_features):
    super().__init__()
    self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size = 1)
    self.branch5x5_1 = BasicConv2d(in_channels, 48, kernel_size = 1)
    self.branch5x5_2 = BasicConv2d(48, 64, kernel_size = 5, padding = 2)
    self.branch3x3dbl_1 = BasicConv2d(in_channels, 64, kernel_size = 1)
    self.branch3x3dbl_2 = BasicConv2d(64, 96, kernel_size = 3, padding = 1)
    self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size = 1)
  def forward(self, x):
    branch1x1 = self.branch1x1(x)
    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)
    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    branch_pool = F.avg_pool2d(x, kernel_size = 3, stride = 1, padding = 1)
    branch_pool = self.branch_pool(branch_pool)
    outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)

inceptionv3 = inception_v3(pretrained = True)
inceptionv3.aux_logits = False
if is_available():
  inceptionv3 = inceptionv3.to("cuda")

class LayerActivations():
  features = []
  def __init__(self, model):
    self.features = []
    self.hook = model.register_forward_hook(self.hook_fn)
  def hook_fn(self, module, input, output):
    self.features.extend(output.view(output.size(0), -1).cpu().data)
  def remove(self):
    self.hook.remove()

trn_features = LayerActivations(inceptionv3.Mixed_7c)
trn_labels = []
for da, la in train_loader:
  _ = inceptionv3(Variable(da.to("cuda")))
  trn_labels.extend(la)
trn_features.remove()
val_features = LayerActivations(inceptionv3.Mixed_7c)
val_labels = []
for da, la in val_loader:
  _ = inceptionv3(Variable(da.to("cuda")))
  val_labels.extend(la)
val_features.remove()

trn_feat_dset = FeaturesDataset(trn_features.features, trn_labels)
val_feat_dset = FeaturesDataset(val_features.features, val_labels)
trn_feat_loader = DataLoader(trn_feat_dset, batch_size = 64, shuffle = True)
val_feat_loader = DataLoader(val_feat_dset, batch_size = 64)

class FullyConnectedModel(nn.Module):
  def __init__(self, in_size, out_size, training = True):
    super().__init__()
    self.fc = nn.Linear(in_size, out_size)
  def forward(self, inp):
    out = F.dropout(inp, training = self.training)
    out = self.fc(out)
    return out

fc_in_size = 131072
fc = FullyConnectedModel(fc_in_size, classes)
if is_available():
  fc = fc.to("cuda")
optimizer = optim.SGD(fc.parameters(), lr = 0.01, momentum = 0.5)

for epoch in range(1, 10):
  epoch_loss, epoch_accuracy = fit(epoch, fc, trn_feat_loader, phase='training')
  val_epoch_loss, val_epoch_accuracy = fit(epoch, fc, val_feat_loader, phase = 'validation')
  train_losses.append(epoch_loss)
  train_accuracy.append(epoch_accuracy)
  val_losses.append(val_epoch_loss)
  val_accuracy.append(val_epoch_accuracy)

# **Creating and Training: DenseNet Model**

In [9]:
class DenseBlock(nn.Sequential):
  def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
    super(DenseBlock, self).__init__()
    for i in range(num_layers):
      layer = DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
      self.add_module('denselayer%d' % (i + 1), layer)

class DenseLayer(nn.Sequential):
  def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
    super(DenseLayer, self).__init__()
    self.add_module('norm.1', nn.BatchNorm2d(num_input_features)),
    self.add_module('relu.1', nn.ReLU(inplace = True)),
    self.add_module('conv.1', nn.Conv2d(num_input_features, bn_size * growth_rate, kernel_size = 1, stride = 1, bias = False)),
    self.add_module('norm.2', nn.BatchNorm2d(bn_size * growth_rate)),
    self.add_module('relu.2', nn.ReLU(inplace = True))
    self.add_module('conv.2', nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size = 3, stride = 1, padding = 1, bias = False)),
    self.drop_rate = drop_rate
  def forward(self, x):
    new_features = super(DenseLayer, self).forward(x)
    if self.drop_rate > 0:
      new_features = F.dropout(new_features, p = self.drop_rate, training = self.training)
    return torch.cat([x, new_features], 1)

densenet = densenet121(pretrained = True).features
if is_available():
  densenet.to("cuda")
for p in densenet.parameters():
  p.requires_grad = False

trn_labels = []
trn_features = []
for d, la in train_loader:
  o = densenet(Variable(d.to("cuda")))
  o = o.view(o.size(0), - 1)
  trn_labels.extend(la)
  trn_features.extend(o.cpu().data)
val_labels = []
val_features = []
for d, la in val_loader:
  o = densenet(Variable(d.cuda()))
  o = o.view(o.size(0), -1)
  val_labels.extend(la)
  val_features.extend(o.cpu().data)

trn_feat_dset = FeaturesDataset(trn_features, trn_labels)
val_feat_dset = FeaturesDataset(val_features, val_labels)
trn_feat_loader = DataLoader(trn_feat_dset, batch_site = 64, shuffle = True, drop_last = True)
val_feat_loader = DataLoader(val_feat_dset, batch_size = 64)

class FullyConnectedModel(nn.Module):
  def __init__(self, in_size, out_size):
    super().__init__()
    self.fc = nn.Linear(in_size, out_size)
  def forward(self, inp):
    out = self.fc(inp)
    return out

fc = FullyConnectedModel(fc_in_size, classes)
if is_available():
  fc = fc.to("cuda")
optimizer = optim.SGD(fc.parameters(), lr = 0.01, momentum = 0.5)
train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []
for epoch in range(1, 10):
  epoch_loss, epoch_accuracy = fit(epoch, fc, trn_feat_loader, phase = 'training')
  val_epoch_loss, val_epoch_accuracy = fit(epoch, fc, val_feat_loader, phase = 'validation')
  train_losses.append(epoch_loss)
  train_accuracy.append(epoch_accuracy)
  val_losses.append(val_epoch_loss)
  val_accuracy.append(val_epoch_accuracy)


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

NameError: ignored

# **Ensembling Model (Resnet, Inception and DenseNet)**

In [ ]:
inception = inception_v3(pretrained = True)
if is_available():
  inception = inception.to("cuda")

trn_labels = []
trn_resnet_features = []
for d, la in train_loader:
  o = resnet_34(Variable(d.to("cuda")))
  o = o.view(o.size(0), -1)
  trn_labels.extend(la)
  trn_resnet_features.extend(o.cpu().data)
val_labels = []
val_resnet_features = []
for d,la in val_loader:
  o = resnet_34(Variable(d.to("cuda")))
  o = o.view(o.size(0), -1)
  val_labels.extend(la)
  val_resnet_features.extend(o.cpu().data)

trn_inception_features = LayerActivations(inception.Mixed_7c)
for da,la in train_loader:
  _ = inception(Variable(da.to("cuda")))
trn_inception_features.remove()
val_inception_features = LayerActivations(inception.Mixed_7c)
for da,la in val_loader:
  _ = inception(Variable(da.to("cuda")))
val_inception_features.remove()

trn_densenet_features = []
for d,la in train_loader:
  o = densenet(Variable(d.to("cuda")))
  o = o.view(o.size(0), -1)
  trn_densenet_features.extend(o.cpu().data)
val_densenet_features = []
for d,la in val_loader:
  o = densenet(Variable(da.to("cuda")))
  o = o.view(o.size(0), -1)
  val_densenet_features.extend(o.cpu().data)

class FeaturesDataset(Dataset):
  def __init__(self, featlst1, featlst2, featlst3, labellst):
    self.featlst1 = featlst1
    self.featlst2 = featlst2
    self.featlst3 = featlst3
    self.labellst = labellst
  def __getitem__(self, index):
    return (self.featlst1[index], self.featlst2[index], self.featlst3[index], self.labellst[index])
  def __len__(self):
    return len(self.labellst)

trn_feat_dset = FeaturesDataset(trn_resnet_features, trn_inception_features.features, trn_densenet_features, trn_labels)
val_feat_dset = FeaturesDataset(val_resnet_features, val_inception_features.features, val_densenet_features, val_labels)
trn_feat_loader = DataLoader(trn_feat_dset, batch_size = 64, shuffle = True)
val_feat_loader = DataLoader(val_feat_dset, batch_size = 64)

class EnsembleModel(nn.Module):
  def __init__(self, out_size, training = True):
    super().__init__()
    self.fc1 = nn.Linear(8192, 512)
    self.fc2 = nn.Linear(131072, 512)
    self.fc3 = nn.Linear(82944, 512)
    self.fc4 = nn.Linear(512, out_size)
  def forward(self, inp1, inp2, inp3):
    out1 = self.fc1(F.dropout(inp1, training = self.training))
    out2 = self.fc2(F.dropout(inp2, training = self.training))
    out3 = self.fc3(F.dropout(inp3, training = self.training))
    out = out1 + out2 + out3
    out = self.fc4(F.dropout(out, training = self.training))
    return out

ensemble = EnsembleModel(2)
if is_available:
  ensemble = ensemble.cuda()

def fit(epoch, model, data_loader, phase = 'training', volatile = False):
  if phase == 'training':
    model.train()
  if phase == 'validation':
    model.eval()
    volatile = True
  running_loss = 0.0
  running_correct = 0
  for batch_idx, (data1, data2, data3, target) in enumerate(data_loader):
    if is_available():
      data1, data2, data3, target = data1.to("cuda"), data2.to("cuda"), data3.to("cuda"), target.to("cuda")
    data1, data2, data3, target = Variable(data1, volatile), Variable(data2, volatile), Variable(data3, volatile), Variable(target, volatile)
    if phase == 'training':
      optimizer.zero_grad()
    output = model(data1, data2, data3)
    loss = F.cross_entropy(output, target)
    running_loss += F.cross_entropy(output, target, size_average = False).item()
    preds = output.data.max(dim = 1, keepdim = True)[1]
    running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
    if phase == 'training':
      loss.backward()
      optimizer.step()
  loss = running_loss / len(data_loader.dataset)
  accuracy = 100. * running_correct / len(data_loader.dataset)
  print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct} / {len(data_loader.dataset)}{accuracy:{10}.{4}}')
  return loss, accuracy

train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []
for epoch in range(1, 10):
  epoch_loss, epoch_accuracy = fit(epoch, ensemble, trn_feat_loader, phase = 'training')
  val_epoch_loss, val_epoch_accuracy = fit(epoch, ensemble, val_feat_loader, phase = 'validation')
  train_losses.append(epoch_loss)
  train_accuracy.append(epoch_accuracy)
  val_losses.append(val_epoch_loss)
  val_accuracy.append(val_epoch_accuracy)
 